In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████████████████  | 264.2 MB 1.3 MB/s eta 0:00:14

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession, column
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,DateType
c_schema = StructType([
            StructField("Scheme Code", StringType(), True),
            StructField("Scheme Name", StringType(), True),
            StructField("ISIN Div Payout", StringType(), True),
            StructField("ISIN Div Reinvestment", StringType(), False),
            StructField("Net Asset Value", StringType(), False),
            StructField("Repurchase Price", StringType(), False),
            StructField("Sale Price", StringType(), False),
            StructField("Date", StringType(), True)
            ])

In [ ]:
df = spark.read.format("csv").option("header", "true")\
            .option("delimiter", ";")\
            .schema(c_schema)\
            .load("/content/spark_input.txt")

In [ ]:
df = df.filter(df['Scheme Name'].isNotNull())
df.show(6)

+-----------+--------------------+---------------+---------------------+---------------+----------------+----------+-----------+
|Scheme Code|         Scheme Name|ISIN Div Payout|ISIN Div Reinvestment|Net Asset Value|Repurchase Price|Sale Price|       Date|
+-----------+--------------------+---------------+---------------------+---------------+----------------+----------+-----------+
|     149383|Axis Multicap Fun...|   INF846K013E0|                 null|          10.01|            null|      null|23-Dec-2021|
|     149383|Axis Multicap Fun...|   INF846K013E0|                 null|          10.01|            null|      null|24-Dec-2021|
|     149383|Axis Multicap Fun...|   INF846K013E0|                 null|          10.01|            null|      null|27-Dec-2021|
|     149383|Axis Multicap Fun...|   INF846K013E0|                 null|          10.05|            null|      null|28-Dec-2021|
|     149383|Axis Multicap Fun...|   INF846K013E0|                 null|          10.05|         

In [ ]:
#update date datatype 
from pyspark.sql.functions import *
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
df = df.withColumn("date",to_date(col('Date'),'dd-MMM-yyyy'))
df.show(5)

root
 |-- Scheme Code: string (nullable = true)
 |-- Scheme Name: string (nullable = true)
 |-- ISIN Div Payout: string (nullable = true)
 |-- ISIN Div Reinvestment: string (nullable = true)
 |-- Net Asset Value: string (nullable = true)
 |-- Repurchase Price: string (nullable = true)
 |-- Sale Price: string (nullable = true)
 |-- date: date (nullable = true)



In [ ]:
df.show(5)

+-----------+--------------------+---------------+---------------------+---------------+----------------+----------+-----------+
|Scheme Code|         Scheme Name|ISIN Div Payout|ISIN Div Reinvestment|Net Asset Value|Repurchase Price|Sale Price|       Date|
+-----------+--------------------+---------------+---------------------+---------------+----------------+----------+-----------+
|     149383|Axis Multicap Fun...|   INF846K013E0|                 null|          10.01|            null|      null|23-Dec-2021|
|     149383|Axis Multicap Fun...|   INF846K013E0|                 null|          10.01|            null|      null|24-Dec-2021|
|     149383|Axis Multicap Fun...|   INF846K013E0|                 null|          10.01|            null|      null|27-Dec-2021|
|     149383|Axis Multicap Fun...|   INF846K013E0|                 null|          10.05|            null|      null|28-Dec-2021|
|     149383|Axis Multicap Fun...|   INF846K013E0|                 null|          10.05|         

In [ ]:
# incremental-load
# check with reference control table to get last updated date

